# Creacion Panel con data de Cortes de Apelaciones

In [31]:
import pandas as pd
import numpy as np
import warnings

# Bokeh Library
from bokeh.io import show, output_notebook, output_file
from bokeh.models import ColumnDataSource, FactorRange, CheckboxButtonGroup, CustomJS, DataTable, DateFormatter, TableColumn
from bokeh.plotting import figure
from bokeh.transform import factor_cmap
from bokeh.models.widgets import Tabs, Panel



In [32]:
# Output to file
output_file('ingresos_terminos.html', 
            title='Ingresos y Terminos por Cortes de Apelaciones - 2015 al 2019')

In [33]:
path_processed = "../data/processed/pjud"

In [34]:
# Carga de datos ...
df_causas = pd.read_feather(f"{path_processed}/consolidated_Fulldata_Causa.feather")

In [35]:
# Carga de datos ...
df_cortes = pd.read_feather(f"{path_processed}/processes_ListadoTribunalesyCortes.feather")
cortes = df_cortes['CORTE'].unique().tolist()

In [36]:
ingresos_por_cortes = []

for (corte, año_ingreso), sub_df in df_causas.groupby(by=['corte','año_ingreso']):
    unique_rol = sub_df['tribunal_rit'].unique()
    ingresos_por_cortes.append([corte, año_ingreso, len(unique_rol)])
    
   
df_ingresos_cortes = pd.DataFrame(ingresos_por_cortes, columns=['corte','año_ingreso','total_causas'])

In [37]:
terminos_por_cortes = []

for (corte, año_termino), sub_df in df_causas.groupby(by=['corte','año_termino']):
    unique_rol = sub_df['tribunal_rit'].unique()
    terminos_por_cortes.append([corte, año_termino, len(unique_rol)])
    
   
df_terminos_cortes = pd.DataFrame(terminos_por_cortes, columns=['corte','año_termino','total_causas'])

In [44]:
# GENERO DICCIONARIO DE DATOS PARA GRAFICAR

cortes = df_causas.corte.unique().tolist()
data_ingresos ={}
data_terminos ={}
data_ingresos['cortes'] = cortes
data_terminos['cortes'] = cortes

years = []
años = range(2015,2020)


for año in años:
    ingreso_año = []
    termino_año = []
    years.append(str(año))
    for corte in cortes:
        criterio_ingresos = f"corte == '{corte}' and año_ingreso == {año}"
        criterio_terminos = f"corte == '{corte}' and año_termino == {año}"
        total_ingreso = df_ingresos_cortes.query(criterio_ingresos).total_causas
        total_termino = df_terminos_cortes.query(criterio_terminos).total_causas
        ingreso_año.append(total_ingreso)
        termino_año.append(total_termino)
    data_ingresos[f'{año}'] = ingreso_año
    data_terminos[f'{año}'] = termino_año

palette = ["#c9d9d3", "#718dbf", "#e84d60", "#CC8dbf", "#AB4d60"]

In [45]:
#output_file("CORTES DE APELACIONES_INGRESOS_CAUSAS_PENALES_2015_AL_2019.html") Si deseo guardar el grafo debo d

# GRAFO INGRESO
x = [ (corte.replace("C.A. DE ",""), year) for corte in cortes for year in years ]
counts = sum(zip(data_ingresos['2015'], data_ingresos['2016'], data_ingresos['2017'], data_ingresos['2018'], data_ingresos['2019']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

plot_ingreso = figure(x_range=FactorRange(*x), plot_height=800, plot_width=1600, title="CORTES DE APELACIONES - INGRESOS CAUSAS PENALES - 2015 AL 2019",
           toolbar_location=None, tools="hover", tooltips="INGRESOS C.A. @x: @counts", y_axis_type="linear", y_range=(10000,150000))

plot_ingreso.vbar(x='x', top='counts', width=0.7, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=years, start=1, end=2))


plot_ingreso.y_range.start = 0
plot_ingreso.x_range.range_padding = 0
plot_ingreso.xaxis.major_label_orientation = 1
plot_ingreso.xgrid.grid_line_color = None


# GRAFO TERMINO
x = [ (corte.replace("C.A. DE ",""), year) for corte in cortes for year in years ]
counts = sum(zip(data_terminos['2015'], data_terminos['2016'], data_terminos['2017'], data_terminos['2018'], data_terminos['2019']), ()) # like an hstack

source = ColumnDataSource(data=dict(x=x, counts=counts))

plot_termino = figure(x_range=FactorRange(*x), plot_height=800, plot_width=1600, title="CORTES DE APELACIONES - TERMINOS CAUSAS PENALES - 2015 AL 2019",
           toolbar_location=None, tools="hover", tooltips="TÉRMINOS C.A. @x: @counts", y_axis_type="linear", y_range=(10000,150000))

plot_termino.vbar(x='x', top='counts', width=0.7, source=source, line_color="white",
       fill_color=factor_cmap('x', palette=palette, factors=years, start=1, end=2))


plot_termino.y_range.start = 0
plot_termino.x_range.range_padding = 0
plot_termino.xaxis.major_label_orientation = 1
plot_termino.xgrid.grid_line_color = None


## Creacion TABLA

In [46]:
data_ingreso = dict(
        corte = df_ingresos_cortes['corte'],
        año = df_ingresos_cortes['año_ingreso'],
        total = df_ingresos_cortes['total_causas']
    )

data_termino = dict(
        corte = df_terminos_cortes['corte'],
        año = df_terminos_cortes['año_termino'],
        total = df_terminos_cortes['total_causas']
    )
source_ingreso = ColumnDataSource(data_ingreso)
source_termino = ColumnDataSource(data_termino)

columns_ingreso = [
        TableColumn(field="corte", title="Corte"),
        TableColumn(field="año", title="Año Ingreso"),
        TableColumn(field="total", title="Total Causas"),
    ]

columns_termino = [
        TableColumn(field="corte", title="Corte"),
        TableColumn(field="año", title="Año Termino"),
        TableColumn(field="total", title="Total Causas"),
    ]

data_table_ingreso = DataTable(source=source_ingreso, columns=columns_ingreso, width=600, height=2800)
data_table_termino = DataTable(source=source_termino, columns=columns_termino, width=600, height=2800)


In [47]:
# Creo panels
ingreso_panel_grafo = Panel(child=plot_ingreso, title='GRÁFICO INGRESOS CAUSAS')
termino_panel_grafo = Panel(child=plot_termino, title='GRÁFICO TERMINOS CAUSAS')
ingreso_panel_table = Panel(child=data_table_ingreso, title='NUMEROS INGRESOS CAUSAS')
termino_panel_table = Panel(child=data_table_termino, title='NUMEROS TERMINOS CAUSAS')

# Assign the panels to Tabs
tabs = Tabs(tabs=[ingreso_panel_grafo, termino_panel_grafo, ingreso_panel_table, termino_panel_table])

# Show the tabbed layout
show(tabs)